In [ ]:
import pickle
import pandas as pd

In [ ]:
ticker = "ES"
interval = 30                       # Assumed to be either 5, 10, 15, 30, 60 minutes
partitions = [0.65, 0.8]

with open("data/futures_ticks.pkl", "rb") as f:
    ticks = pickle.load(f)[ticker]
ticker += "_temp"       
futures_df = pd.read_csv(f"./data/S&P500 Mini/{ticker}.csv", index_col='t', parse_dates=True)

resampled_o = futures_df['o'].resample(f"{interval}min").first()
resampled_h = futures_df['h'].resample(f"{interval}min").max()
resampled_l = futures_df['l'].resample(f"{interval}min").min()
resampled_c = futures_df['c'].resample(f"{interval}min").last()
resampled_v = futures_df['v'].resample(f"{interval}min").sum()
main_df = pd.DataFrame(
    {
        'o': resampled_o,
        'h': resampled_h,
        'l': resampled_l,
        'c': resampled_c,
        'v': resampled_v,
    }
)
main_df["delta"] = main_df["c"].diff()
main_df["fwd_r_u"] = ((main_df["h"] - main_df["o"]) / ticks).shift(-1)
main_df["fwd_r_d"] = ((main_df["o"] - main_df["l"]) / ticks).shift(-1)
main_df["fwd_r_o"] = main_df["fwd_r_u"] + main_df["fwd_r_d"]

In [30]:
main_df["delta"] = main_df["c"].diff()

main_df["fwd_r_u"] = ((main_df["h"] - main_df["o"]) / ticks).shift(-1)
main_df["fwd_r_d"] = ((main_df["o"] - main_df["l"]) / ticks).shift(-1)
main_df["fwd_r_o"] = main_df["fwd_r_u"] + main_df["fwd_r_d"]

In [ ]:
# Visualization: plots on range up, down, overall PDF and CDF